# Pipeline

In this section I will ensamble all the step to create the proper Pipeline

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt
from pathlib import Path
import os
import pipeline as pp

In [2]:
# Make a list of calibration images
images = glob.glob('../test_images/*.jpg')
i = 0

# Making output directory
output_dir = "../output_images/pipeline/"
Path(output_dir).mkdir(parents=True, exist_ok=True)

for fname in images:
    # original
    img = mpimg.imread(fname)

    # undistort image
    img_undist = pp.undistort_image(img)

    # def pipeline(img):
    # bitmap with streigh guides
    bitmap = pp.threshold(img_undist)

    # eagle view with streigh guides
    img_eagle_eye = pp.eagle_eye(bitmap)

    x_half = int(img_eagle_eye.shape[1] / 2)
    y_half = int(img_eagle_eye.shape[0] / 2)
    bottom_half = img_eagle_eye[y_half:, :]

    # computing poly_fit using fit_poly in pipeline
    leftx, lefty, rightx, righty = pp.find_lane_pixels_window(bottom_half, nwindows=10, margin=100, minpix=40)
    left_fit, right_fit, left_fitx, right_fitx, ploty = pp.fit_poly(bottom_half.shape, leftx, lefty, rightx, righty)

    # Init image to plot lane
    lines_img = np.zeros_like(img)

    # Plot the are between the fitting lines
    left_pts = np.transpose(np.array( [left_fitx, y_half + ploty]))
    right_pts = np.transpose(np.array( [right_fitx, y_half + ploty]))
    left_line_window = np.array([np.transpose(np.vstack([left_fitx, y_half + ploty]))])
    right_line_window = np.array([np.flipud(np.transpose(np.vstack([right_fitx, 
                                y_half + ploty])))])
    lane_area = np.hstack((left_line_window, right_line_window))
    cv2.fillPoly(lines_img, np.int_([lane_area]), (0,255, 0))


    # Plot the polynomial lines onto the image
    thikness = 20
    lines_img = cv2.polylines(lines_img, np.int32([left_pts]), False, (255, 0, 0), thikness) 
    lines_img = cv2.polylines(lines_img, np.int32([right_pts]), False, (0, 0, 255), thikness) 

    # apply inverse of the eagle eye transformation
    img_eagle_eye = pp.eagle_eye_inv(lines_img)

    # combine with original image
    combined_img = cv2.addWeighted(img_undist, 1, img_eagle_eye, 0.6, 0)

    # tranfomation to real space
    left_fit_real, right_fit_real = pp.fit_to_real_space(left_fit, right_fit)
    left_fitx_real = left_fitx * pp.xm_per_pix
    right_fitx_real = right_fitx * pp.xm_per_pix
    ploty_real = ploty * pp.ym_per_pix

    position = pp.car_position(img, ploty_real, left_fit_real, right_fit_real)
    # computing curvature
    left_curverad, right_curverad = pp.measure_curvature(ploty_real, left_fit_real, right_fit_real)
    curve_radious = (left_curverad + right_curverad) / 2

    string_curvature = "The curvature radios is " + "{:.2f}".format(curve_radious) + "m"
    string_position = "The car position is " + "{:.2f}".format(position) + "m from the center"
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfCurvatureText = (100,100)
    bottomLeftCornerOfPositionText = (100,150)
    fontScale              = 1
    fontColor              = (255,255,255)
    lineType               = 2
    cv2.putText(combined_img, string_curvature, bottomLeftCornerOfCurvatureText, font, fontScale, fontColor, lineType)
    cv2.putText(combined_img, string_position, bottomLeftCornerOfPositionText, font, fontScale, fontColor, lineType)

    f = plt.figure(i)
    i = i + 1
    plt.imshow(combined_img)

    f.savefig(output_dir + os.path.basename(fname))

After looking the full assemble we can see that the fitting in the images is done corretly:

![Lanes Image](../output_images/pipeline/test2.jpg) 

![Lanes Image](../output_images/pipeline/straight_lines1.jpg) 

So I procede to write the final version in the pipeline.py and process the video

In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import pipeline as pp

global left_fit
global right_fit

pipeline = pp.Pipeline()

def process_image(image):
    result = pipeline.step(image)

    return result

clip1 = VideoFileClip("../project_video.mp4")
video_output = '../output_images/pipeline/project_video.mp4'

white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(video_output, audio=False)

t:   0%|          | 2/1260 [00:00<01:30, 13.89it/s, now=None]Moviepy - Building video ../output_images/pipeline/project_video.mp4.
Moviepy - Writing video ../output_images/pipeline/project_video.mp4

Moviepy - Done !
Moviepy - video ready ../output_images/pipeline/project_video.mp4
Wall time: 2min 1s
